In [2]:
import pandas as pd

In [3]:
train = pd.read_csv("./data/labeledTrainData.tsv", header=0, 
                     delimiter="\t", quoting=3)

test = pd.read_csv( "./data/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv("./data/unlabeledTrainData.tsv", header=0, 
                              delimiter="\t", quoting=3 )

In [4]:
# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews, and %d unlabeled reviews\n" % (train["review"].size, test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [5]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [6]:
train['review'][0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

接下来对数据进行清洗，和Part 1差不多，不过有些不一样的地方。首先，训练word2vec的时候，最好不要去除stop words，因为word2vec算法中，更多的词汇能产生更高质量的词向量，所以我们提供一个可选项。另外，最好不要去除数字：

In [7]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review, remove_stopwords=False):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    
    # 1. Remove HTML
    review_text = BeautifulSoup(review, 'lxml').get_text()
      
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()

    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    
    # 5. Return a list of words
    return(words)

现在，我们想要规定好输入的格式。输入Word2vec的是单个句子，一个句子是一个list，由词组成。换句话说，输入格式是a list of lists。
想要把段落分割为句子并不是一件直观的事情。英语句子的结尾可以是"?", "!", """, or ".", 而空格和大小写也靠不住。所以，我们将使用NLTK中的punkt标记生成器来进行句子分割。我们先得下载NLTK，然后用nltk.download()来下载相关的训练文件，来训练punkt。

In [10]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
#nltk.download()   


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [11]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, remove_stopwords ))

    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [12]:
# tokenizer只负责将paragraph分割为多个sentence
# 对于每个sentence，review_to_wordlist负责对一个sentence进行清洗
s1 = review_to_sentences(train['review'][0], tokenizer)
# 输出第一个sentence的list

In [14]:
s1[1]

['maybe',
 'i',
 'just',
 'want',
 'to',
 'get',
 'a',
 'certain',
 'insight',
 'into',
 'this',
 'guy',
 'who',
 'i',
 'thought',
 'was',
 'really',
 'cool',
 'in',
 'the',
 'eighties',
 'just',
 'to',
 'maybe',
 'make',
 'up',
 'my',
 'mind',
 'whether',
 'he',
 'is',
 'guilty',
 'or',
 'innocent']

把这个函数用来处理数据上，准备给word2vec的输入

In [15]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/Users/james/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/james/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/Users/james/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/james/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/james/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client lik

In [16]:
# Check how many sentences we have in total - should be around 850,000+
len(sentences)

795538

In [17]:
print(sentences[0])

print(sentences[1])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


#### 训练和保存模型
###### 在训练模型的时候，有几个参数会影响运行时间和生成的模型效果：
###### Architecture: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.
###### Training algorithm: Hierarchical softmax (default) or negative sampling. For us, the default worked well.
###### Downsampling of frequent words: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.
###### Word vector dimensionality: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.
###### Context / window size: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).
###### Worker threads: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.
###### Minimum word count: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.
###### 设定参数并不容易，但一旦设定好，训练模型就简单多了：

In [18]:
# Import the built-in logging module and configure it 
# so that Word2Vec creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, 
            size=num_features, min_count = min_word_count, 
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "model/300features_40minwords_10context"
model.save(model_name)

2019-04-03 11:05:43,899 : INFO : 'pattern' package not found; tag filters are not available for English
2019-04-03 11:05:43,916 : INFO : collecting all words and their counts
2019-04-03 11:05:43,917 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-04-03 11:05:43,996 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2019-04-03 11:05:44,066 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types


Training model...


2019-04-03 11:05:44,140 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2019-04-03 11:05:44,226 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2019-04-03 11:05:44,298 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2019-04-03 11:05:44,379 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2019-04-03 11:05:44,467 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2019-04-03 11:05:44,547 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2019-04-03 11:05:44,637 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2019-04-03 11:05:44,714 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2019-04-03 11:05:44,781 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 

2019-04-03 11:05:49,893 : INFO : PROGRESS: at sentence #750000, processed 16771230 words, keeping 120295 word types
2019-04-03 11:05:49,978 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2019-04-03 11:05:50,070 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2019-04-03 11:05:50,165 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2019-04-03 11:05:50,252 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2019-04-03 11:05:50,303 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2019-04-03 11:05:50,304 : INFO : Loading a fresh vocabulary
2019-04-03 11:05:50,394 : INFO : effective_min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2019-04-03 11:05:50,395 : INFO : effective_min_count=40 leaves 17238940 word corpus (96% of original 17798082, drops 559

2019-04-03 11:06:43,441 : INFO : EPOCH 3 - PROGRESS: at 34.99% examples, 547918 words/s, in_qsize 7, out_qsize 0
2019-04-03 11:06:44,451 : INFO : EPOCH 3 - PROGRESS: at 39.64% examples, 552438 words/s, in_qsize 7, out_qsize 0
2019-04-03 11:06:45,455 : INFO : EPOCH 3 - PROGRESS: at 44.15% examples, 554916 words/s, in_qsize 7, out_qsize 0
2019-04-03 11:06:46,465 : INFO : EPOCH 3 - PROGRESS: at 49.04% examples, 561076 words/s, in_qsize 8, out_qsize 0
2019-04-03 11:06:47,468 : INFO : EPOCH 3 - PROGRESS: at 53.77% examples, 564173 words/s, in_qsize 7, out_qsize 0
2019-04-03 11:06:48,479 : INFO : EPOCH 3 - PROGRESS: at 58.51% examples, 567532 words/s, in_qsize 6, out_qsize 1
2019-04-03 11:06:49,484 : INFO : EPOCH 3 - PROGRESS: at 63.05% examples, 568198 words/s, in_qsize 7, out_qsize 0
2019-04-03 11:06:50,486 : INFO : EPOCH 3 - PROGRESS: at 67.63% examples, 569315 words/s, in_qsize 7, out_qsize 1
2019-04-03 11:06:51,493 : INFO : EPOCH 3 - PROGRESS: at 71.54% examples, 564841 words/s, in_qsiz

2019-04-03 11:07:40,877 : INFO : not storing attribute vectors_norm
2019-04-03 11:07:40,878 : INFO : not storing attribute cum_table


FileNotFoundError: [Errno 2] No such file or directory: 'model/300features_40minwords_10context'

### 探索模型结果
训练结束后，查看一下75000个评论的训练结果吧。doesnt_match函数会推断在一个集合里，哪一个单词与其他单词最不相似：

In [19]:
model.doesnt_match('man woman child kitchen'.split())

/Users/james/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/Users/james/anaconda3/lib/python3.6/site-packages/gensim/models/keyedvectors.py:858: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'kitchen'

In [20]:
model.doesnt_match('france england germany berlin'.split())

/Users/james/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/Users/james/anaconda3/lib/python3.6/site-packages/gensim/models/keyedvectors.py:858: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'berlin'

In [21]:
model.doesnt_match('paris berlin london austria'.split())

/Users/james/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/Users/james/anaconda3/lib/python3.6/site-packages/gensim/models/keyedvectors.py:858: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'paris'

In [22]:
model.most_similar('man')

/Users/james/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.6312069892883301),
 ('lady', 0.6071215271949768),
 ('lad', 0.6062389016151428),
 ('guy', 0.5259661674499512),
 ('soldier', 0.5251173377037048),
 ('monk', 0.5240025520324707),
 ('men', 0.5216838121414185),
 ('businessman', 0.516812801361084),
 ('person', 0.5152419805526733),
 ('chap', 0.5070861577987671)]

In [23]:
model.most_similar('queen')

/Users/james/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('princess', 0.6636403799057007),
 ('bride', 0.6179916858673096),
 ('dame', 0.6162452101707458),
 ('victoria', 0.6026084423065186),
 ('duchess', 0.5881589651107788),
 ('showgirl', 0.5862511396408081),
 ('stepmother', 0.5838395357131958),
 ('goddess', 0.5795204639434814),
 ('latifah', 0.5751671195030212),
 ('belle', 0.5746701955795288)]